## Named Entity Recognition

In [11]:
import os 

import numpy as np
import pandas as pd 
import tensorflow as tf
import keras

tf.keras.utils.set_random_seed(33)

In [5]:
def get_vocab(vocab_path, tags_path):
    vocab = {}

    with open(vocab_path) as f:
        for i, l in enumerate(f.read().splitlines()):
            vocab[l] = i
        
    vocab['<PAD>'] = len(vocab)
    tag_map = {}

    with open(tags_path) as f:
        for i, t in enumerate(f.read().splitlines()):
            tag_map[t] = i
        
    return vocab, tag_map


def get_params(vocab, tag_map, sentence_file, labels_file):
    sentences = []
    labels = []

    with open(sentence_file) as f:
        for sentence in f.read().splitlines():
            s = [vocab[token] if token in vocab
                 else vocab['UNK']
                 for token in sentence.split('')]
            
            # Tokenize and append all the sentences
            sentence.append(s)
    
    with open(labels_file) as f:
        for sentence in f.read().splitlines():
            l = [tag_map[label] for label in sentence.split(' ')] # I added plus 1 here
            labels.append(l) 
    return sentences, labels, len(sentences)



In [8]:
# display original kaggle data
data = pd.read_csv("data/ner_dataset.csv", encoding = "ISO-8859-1") 
train_sents = open('data/small/train/sentences.txt', 'r').readline()
train_labels = open('data/small/train/labels.txt', 'r').readline()
print('SENTENCE:', train_sents)
print('SENTENCE LABEL:', train_labels)
print('ORIGINAL DATA:\n', data.head())
del(data, train_sents, train_labels)

SENTENCE: Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .

SENTENCE LABEL: O O O O O O B-geo O O O O O B-geo O O O O O B-gpe O O O O O

ORIGINAL DATA:
     Sentence #           Word  POS Tag
0  Sentence: 1      Thousands  NNS   O
1          NaN             of   IN   O
2          NaN  demonstrators  NNS   O
3          NaN           have  VBP   O
4          NaN        marched  VBN   O


In [9]:
def load_data(file_path):
    with open(file_path,'r') as file:
        data = np.array([line.strip() for line in file.readlines()])
    return data
    

In [10]:
train_sentences = load_data('data/large/train/sentences.txt')
train_labels = load_data('data/large/train/labels.txt')

val_sentences = load_data('data/large/val/sentences.txt')
val_labels = load_data('data/large/val/labels.txt')

test_sentences = load_data('data/large/test/sentences.txt')
test_labels = load_data('data/large/test/labels.txt')

In [13]:
def get_sentence_vectorizer(sentences):
    tf.keras.utils.set_random_seed(33)
   
    sentence_vectorizer = keras.layers.TextVectorization(standardize=None)
    sentence_vectorizer.adapt(sentences)
    vocab = sentence_vectorizer.get_vocabulary()
    
    return sentence_vectorizer, vocab

In [24]:
print(f"Sentence: {train_sentences[0]}")
print(f"Labels: {train_labels[0]}")

Sentence: Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .
Labels: O O O O O O B-geo O O O O O B-geo O O O O O B-gpe O O O O O


In [15]:
def get_tags(labels):
    tag_set = set() # Define an empty set
    for el in labels:
        for tag in el.split(" "):
            tag_set.add(tag)
    tag_list = list(tag_set) 
    tag_list.sort()
    return tag_list

In [16]:
tags = get_tags(train_labels)
print(tags)

['B-art', 'B-eve', 'B-geo', 'B-gpe', 'B-nat', 'B-org', 'B-per', 'B-tim', 'I-art', 'I-eve', 'I-geo', 'I-gpe', 'I-nat', 'I-org', 'I-per', 'I-tim', 'O']


In [17]:
def make_tag_map(tags):
    tag_map = {}
    for i,tag in enumerate(tags):
        tag_map[tag] = i 
    return tag_map

In [18]:
tag_map = make_tag_map(tags)
print(tag_map)

{'B-art': 0, 'B-eve': 1, 'B-geo': 2, 'B-gpe': 3, 'B-nat': 4, 'B-org': 5, 'B-per': 6, 'B-tim': 7, 'I-art': 8, 'I-eve': 9, 'I-geo': 10, 'I-gpe': 11, 'I-nat': 12, 'I-org': 13, 'I-per': 14, 'I-tim': 15, 'O': 16}


### Padding the Labels

In [19]:
def label_vectorizer(labels, tag_map):
    label_ids = []

    for element in labels:
        tokens = element.split()
        element_ids = [tag_map[token] for token in tokens]
        label_ids.append(element_ids)

    label_ids = keras.preprocessing.sequence.pad_sequences(label_ids, padding='post', value=-1)

    return label_ids


In [20]:
print(f"Sentence: {train_sentences[5]}")
print(f"Labels: {train_labels[5]}")
print(f"Vectorized labels: {label_vectorizer([train_labels[5]], tag_map)}")

Sentence: The party is divided over Britain 's participation in the Iraq conflict and the continued deployment of 8,500 British troops in that country .
Labels: O O O O O B-gpe O O O O B-geo O O O O O O O B-gpe O O O O O
Vectorized labels: [[16 16 16 16 16  3 16 16 16 16  2 16 16 16 16 16 16 16  3 16 16 16 16 16]]


In [21]:
def generate_dataset(sentences, labels, sentence_vectorizer, tag_map):
    sentences_ids = sentence_vectorizer(sentences)
    labels_ids = label_vectorizer(labels, tag_map = tag_map)
    dataset = tf.data.Dataset.from_tensor_slices((sentences_ids, labels_ids))
    return dataset

In [27]:
def NER(len_tags, vocab_size, embedding_dim=50):
    
    model = tf.keras.Sequential(name='sequential')

    model.add(tf.keras.layers.Embedding(input_dim=vocab_size + 1, output_dim=embedding_dim, mask_zero=True))
    model.add(tf.keras.layers.LSTM(units=embedding_dim, return_sequences=True))

    model.add(tf.keras.layers.Dense(units=len_tags, activation=tf.nn.log_softmax))

    return model


In [28]:
def masked_loss(y_true, y_pred):
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

    loss = loss_fn(y_true, y_pred)

    mask = tf.math.not_equal(y_true, -1)
    loss = tf.where(mask, loss, 0)

    loss = tf.reduce_mean(loss)

    return loss


In [ ]:
def masked_accuracy(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32) 
    mask = tf.cast(tf.math.not_equal(y_true, -1), tf.float32) 
    y_pred_class = tf.math.argmax(y_pred, axis=-1, output_type=tf.int32) 
    y_pred_class = tf.cast(y_pred_class, tf.float32) 
    matches_true_pred  = tf.equal(y_true, y_pred_class)
    matches_true_pred = tf.cast(matches_true_pred , tf.float32) 
    matches_true_pred *= mask
    masked_acc = tf.reduce_sum(matches_true_pred) / tf.reduce_sum(mask)

    return masked_acc
